### Get stations Data from Applied Climate Information System

In [8]:
import requests
import csv
import time

# Define bounding box
min_lat = 37.5
max_lat = 41.7
min_lon = -76.0
max_lon = -72.5

# --- STEP 1: Find NJ stations reporting precipitation ---
meta_url = "https://data.rcc-acis.org/StnMeta"

meta_payload = {
    "state": "NJ",
    "meta": ["name", "state", "sids", "ll", "valid_daterange"],
    "elems": ["pcpn"]
}

try:
    meta_response = requests.post(meta_url, json=meta_payload)
    meta_response.raise_for_status()
    stations = meta_response.json()["meta"]
except requests.exceptions.RequestException as e:
    print(f"Error fetching station metadata: {e}")
    exit(1)

print(f"Total stations found in NJ: {len(stations)}")

# Debug: Look at sample station data
print("\nSample station data:")
for i, stn in enumerate(stations[:3]):
    print(f"Station {i}: {stn}")
    if i >= 2:
        break

# Prepare CSV output
csv_file = open("nj_precip_coop_stations.csv", mode="w", newline="")
csv_writer = csv.writer(csv_file)
csv_writer.writerow([
    "StationName", "StationID", "Date", "Precip(in)", "TimeObs", "Flags"
])

station_count = 0
skipped_no_coords = 0
skipped_out_of_bounds = 0
skipped_no_coop = 0

# --- STEP 2: Loop through each station ---
for stn in stations:
    # Check if station has lat/lon coordinates
    if "ll" not in stn or not stn["ll"]:
        skipped_no_coords += 1
        continue
        
    # Check if lat/lon is in your bounding box
    try:
        lon, lat = stn["ll"]
        if not (min_lat <= lat <= max_lat and min_lon <= lon <= max_lon):
            skipped_out_of_bounds += 1
            continue
    except (ValueError, TypeError):
        skipped_no_coords += 1
        continue

    # Look for a COOP network ID (idType 7) and ACIS ID (idType 2)
    coop_sid = None
    acis_sid = None
    
    if "sids" in stn and stn["sids"]:
        for s in stn["sids"]:
            parts = s.split()
            if len(parts) == 2:
                sid_value, sid_type = parts
                if sid_type == "7":
                    coop_sid = sid_value
                if sid_type == "2":
                    acis_sid = sid_value

    if coop_sid is None or acis_sid is None:
        skipped_no_coop += 1
        continue

    # --- Build StnData request ---
    payload = {
        "sid": acis_sid,
        "sdate": "1980-01-01",
        "edate": "2024-12-31",
        "meta": ["name", "state", "sids"],
        "elems": [
            {
                "name": "pcpn",
                "interval": "dly",
                "duration": "dly",
                "add": "t,i",
                "prec": 2
            }
        ]
    }

    data_url = "https://data.rcc-acis.org/StnData"
    
    try:
        resp = requests.post(data_url, json=payload)
        resp.raise_for_status()
        data = resp.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {stn.get('name', 'Unknown')}: {e}")
        continue

    # --- Write to CSV ---
    if "data" in data and data["data"]:
        station_name = data["meta"]["name"]
        for row in data["data"]:
            # Each row = [date, value, timeObs, flags]
            date = row[0] if len(row) > 0 else ""
            precip = row[1] if len(row) > 1 else ""
            time_obs = row[2] if len(row) > 2 else ""
            flags = row[3] if len(row) > 3 else ""

            csv_writer.writerow([
                station_name, acis_sid, date, precip, time_obs, flags
            ])
        station_count += 1
        print(f"Downloaded: {station_name} (COOP: {coop_sid}, ACIS: {acis_sid})")
    else:
        print(f"No data for {stn.get('name', 'Unknown station')}")

    # Be respectful to the API
    time.sleep(0.1)

csv_file.close()

print(f"\nDone. Data saved for {station_count} stations.")
print(f"Skipped {skipped_no_coords} stations without coordinates")
print(f"Skipped {skipped_out_of_bounds} stations outside bounding box")
print(f"Skipped {skipped_no_coop} stations without COOP/ACIS IDs")
print(f"Total stations processed: {len(stations)}")

Total stations found in NJ: 877

Sample station data:
Station 0: {'ll': [-74.91667, 38.98333], 'sids': ['13775 1', 'USW00013775 6', 'USW00013775 32'], 'state': 'NJ', 'valid_daterange': [['1945-02-04', '1945-12-07']], 'name': 'WILDWOOD NAS'}
Station 1: {'ll': [-74.88333, 38.95], 'sids': ['13756 1', 'USW00013756 6', 'USW00013756 32'], 'state': 'NJ', 'valid_daterange': [['1945-02-04', '1946-04-30']], 'name': 'CAPE MAY NAS'}
Station 2: {'ll': [-74.93611, 38.95341], 'sids': ['281351 2', 'USC00281351 6', 'CPMN4 7'], 'state': 'NJ', 'valid_daterange': [['1894-01-01', '2025-07-12']], 'name': 'CAPE MAY 2 NW'}
Downloaded: CAPE MAY 2 NW (COOP: CPMN4, ACIS: 281351)
Downloaded: CAPE MAY CT HOUSE 4 N (COOP: CMCN4, ACIS: 281349)
Downloaded: BELLEPLAIN STA FOREST (COOP: WDBN4, ACIS: 280690)
Downloaded: ATLANTIC CITY MARINA (COOP: ATLN4, ACIS: 280325)
Downloaded: MILLVILLE MUNICIPAL AIRPORT (COOP: MIV, ACIS: 285581)
Downloaded: ATLANTIC CITY INTL AP (COOP: ACY, ACIS: 280311)
Downloaded: MAYS LANDING 1 W

In [9]:
import requests
import csv
import time

# Define bounding box
min_lat = 37.5
max_lat = 41.7
min_lon = -76.0
max_lon = -72.5

# --- STEP 1: Find NJ stations reporting precipitation ---
meta_url = "https://data.rcc-acis.org/StnMeta"

meta_payload = {
    "state": "NJ",
    "meta": ["name", "state", "sids", "ll", "valid_daterange"],
    "elems": ["pcpn"]
}

try:
    meta_response = requests.post(meta_url, json=meta_payload)
    meta_response.raise_for_status()
    stations = meta_response.json()["meta"]
except requests.exceptions.RequestException as e:
    print(f"Error fetching station metadata: {e}")
    exit(1)

print(f"Total stations found in NJ: {len(stations)}")

# Debug: Look at sample station data
print("\nSample station data:")
for i, stn in enumerate(stations[:3]):
    print(f"Station {i}: {stn}")
    if i >= 2:
        break

# Prepare CSV output
csv_file = open("nj_precipitation_data_1980_2024.csv", mode="w", newline="")
csv_writer = csv.writer(csv_file)
csv_writer.writerow([
    "StationName", "StationID", "Date", "Precip(in)", "TimeObs", "Flags"
])

station_count = 0
skipped_no_coords = 0
skipped_out_of_bounds = 0
skipped_no_coop = 0

# --- STEP 2: Loop through each station ---
for stn in stations:
    # Check if station has lat/lon coordinates
    if "ll" not in stn or not stn["ll"]:
        skipped_no_coords += 1
        continue
        
    # Check if lat/lon is in your bounding box
    try:
        lon, lat = stn["ll"]
        if not (min_lat <= lat <= max_lat and min_lon <= lon <= max_lon):
            skipped_out_of_bounds += 1
            continue
    except (ValueError, TypeError):
        skipped_no_coords += 1
        continue

    # Look for a COOP network ID (idType 7) and ACIS ID (idType 2)
    coop_sid = None
    acis_sid = None
    
    if "sids" in stn and stn["sids"]:
        for s in stn["sids"]:
            parts = s.split()
            if len(parts) == 2:
                sid_value, sid_type = parts
                if sid_type == "7":
                    coop_sid = sid_value
                if sid_type == "2":
                    acis_sid = sid_value

    if coop_sid is None or acis_sid is None:
        skipped_no_coop += 1
        continue

    # --- Build StnData request ---
    payload = {
        "sid": acis_sid,
        "sdate": "1980-01-01",
        "edate": "2024-12-31",
        "meta": ["name", "state", "sids"],
        "elems": [
            {
                "name": "pcpn",
                "interval": "dly",
                "duration": "dly",
                "add": "t,i",
                "prec": 2
            }
        ]
    }

    data_url = "https://data.rcc-acis.org/StnData"
    
    try:
        resp = requests.post(data_url, json=payload)
        resp.raise_for_status()
        data = resp.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {stn.get('name', 'Unknown')}: {e}")
        continue

    # --- Write to CSV ---
    if "data" in data and data["data"]:
        station_name = data["meta"]["name"]
        for row in data["data"]:
            # Each row = [date, [precip_value, time_obs, flags]]
            date = row[0] if len(row) > 0 else ""
            
            # Parse the precipitation data (it's in format [value, time, flags])
            if len(row) > 1 and row[1] is not None:
                precip_data = row[1]
                if isinstance(precip_data, list) and len(precip_data) > 0:
                    # Extract the actual precipitation value
                    precip_value = precip_data[0]
                    time_obs = precip_data[1] if len(precip_data) > 1 else ""
                    flags = precip_data[2] if len(precip_data) > 2 else ""
                    
                    # Clean up the precipitation value
                    if precip_value == "M" or precip_value == "T":
                        # "M" = Missing, "T" = Trace (keep as is)
                        precip_clean = precip_value
                    else:
                        try:
                            # Convert to float and back to string to clean format
                            precip_clean = str(float(precip_value))
                        except (ValueError, TypeError):
                            precip_clean = precip_value
                else:
                    precip_clean = ""
                    time_obs = ""
                    flags = ""
            else:
                precip_clean = ""
                time_obs = ""
                flags = ""

            csv_writer.writerow([
                station_name, acis_sid, date, precip_clean, time_obs, flags
            ])
        station_count += 1
        print(f"Downloaded: {station_name} (COOP: {coop_sid}, ACIS: {acis_sid})")
    else:
        print(f"No data for {stn.get('name', 'Unknown station')}")

    # Be respectful to the API
    time.sleep(0.1)

csv_file.close()

print(f"\nDone. Data saved for {station_count} stations.")
print(f"Skipped {skipped_no_coords} stations without coordinates")
print(f"Skipped {skipped_out_of_bounds} stations outside bounding box")
print(f"Skipped {skipped_no_coop} stations without COOP/ACIS IDs")
print(f"Total stations processed: {len(stations)}")

Total stations found in NJ: 877

Sample station data:
Station 0: {'ll': [-74.91667, 38.98333], 'sids': ['13775 1', 'USW00013775 6', 'USW00013775 32'], 'state': 'NJ', 'valid_daterange': [['1945-02-04', '1945-12-07']], 'name': 'WILDWOOD NAS'}
Station 1: {'ll': [-74.88333, 38.95], 'sids': ['13756 1', 'USW00013756 6', 'USW00013756 32'], 'state': 'NJ', 'valid_daterange': [['1945-02-04', '1946-04-30']], 'name': 'CAPE MAY NAS'}
Station 2: {'ll': [-74.93611, 38.95341], 'sids': ['281351 2', 'USC00281351 6', 'CPMN4 7'], 'state': 'NJ', 'valid_daterange': [['1894-01-01', '2025-07-12']], 'name': 'CAPE MAY 2 NW'}
Downloaded: CAPE MAY 2 NW (COOP: CPMN4, ACIS: 281351)
Downloaded: CAPE MAY CT HOUSE 4 N (COOP: CMCN4, ACIS: 281349)
Downloaded: BELLEPLAIN STA FOREST (COOP: WDBN4, ACIS: 280690)
Downloaded: ATLANTIC CITY MARINA (COOP: ATLN4, ACIS: 280325)
Downloaded: MILLVILLE MUNICIPAL AIRPORT (COOP: MIV, ACIS: 285581)
Downloaded: ATLANTIC CITY INTL AP (COOP: ACY, ACIS: 280311)
Downloaded: MAYS LANDING 1 W

In [10]:
import requests
import csv
import time

# Define bounding box
min_lat = 37.5
max_lat = 41.7
min_lon = -76.0
max_lon = -72.5

# --- STEP 1: Find NJ stations reporting precipitation ---
meta_url = "https://data.rcc-acis.org/StnMeta"

meta_payload = {
    "state": "NJ",
    "meta": ["name", "state", "sids", "ll", "valid_daterange"],
    "elems": ["pcpn"]
}

try:
    meta_response = requests.post(meta_url, json=meta_payload)
    meta_response.raise_for_status()
    stations = meta_response.json()["meta"]
except requests.exceptions.RequestException as e:
    print(f"Error fetching station metadata: {e}")
    exit(1)

print(f"Total stations found in NJ: {len(stations)}")

# Debug: Look at sample station data
print("\nSample station data:")
for i, stn in enumerate(stations[:3]):
    print(f"Station {i}: {stn}")
    if i >= 2:
        break

# Prepare CSV output
csv_file = open("nj_last_precipitation_data_1980_2024.csv", mode="w", newline="")
csv_writer = csv.writer(csv_file)
csv_writer.writerow([
    "StationName", "StationID", "Date", "Precip(in)", "TimeObs", "Flags"
])

station_count = 0
skipped_out_of_bounds = 0
skipped_no_coop = 0

# --- STEP 2: Loop through each station ---
for stn in stations:
    # Look for a COOP network ID (idType 7) and ACIS ID (idType 2) first
    coop_sid = None
    acis_sid = None
    
    if "sids" in stn and stn["sids"]:
        for s in stn["sids"]:
            parts = s.split()
            if len(parts) == 2:
                sid_value, sid_type = parts
                if sid_type == "7":
                    coop_sid = sid_value
                if sid_type == "2":
                    acis_sid = sid_value

    # Skip if no COOP or ACIS ID
    if coop_sid is None or acis_sid is None:
        skipped_no_coop += 1
        continue

    # Check coordinates only if they exist
    has_coords = "ll" in stn and stn["ll"]
    if has_coords:
        try:
            lon, lat = stn["ll"]
            # Check if within bounding box
            if not (min_lat <= lat <= max_lat and min_lon <= lon <= max_lon):
                skipped_out_of_bounds += 1
                continue
        except (ValueError, TypeError):
            # Treat as no coordinates if parsing fails
            has_coords = False
    
    # If no coordinates, we'll still include the station since it's COOP
    if not has_coords:
        print(f"Including COOP station without coordinates: {stn.get('name', 'Unknown')} ({coop_sid})")

    # --- Build StnData request ---
    payload = {
        "sid": acis_sid,
        "sdate": "1980-01-01",
        "edate": "2024-12-31",
        "meta": ["name", "state", "sids"],
        "elems": [
            {
                "name": "pcpn",
                "interval": "dly",
                "duration": "dly",
                "add": "t,i",
                "prec": 2
            }
        ]
    }

    data_url = "https://data.rcc-acis.org/StnData"
    
    try:
        resp = requests.post(data_url, json=payload)
        resp.raise_for_status()
        data = resp.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {stn.get('name', 'Unknown')}: {e}")
        continue

    # --- Write to CSV ---
    if "data" in data and data["data"]:
        station_name = data["meta"]["name"]
        for row in data["data"]:
            # Each row = [date, [precip_value, time_obs, flags]]
            date = row[0] if len(row) > 0 else ""
            
            # Parse the precipitation data (it's in format [value, time, flags])
            if len(row) > 1 and row[1] is not None:
                precip_data = row[1]
                if isinstance(precip_data, list) and len(precip_data) > 0:
                    # Extract the actual precipitation value
                    precip_value = precip_data[0]
                    time_obs = precip_data[1] if len(precip_data) > 1 else ""
                    flags = precip_data[2] if len(precip_data) > 2 else ""
                    
                    # Clean up the precipitation value
                    if precip_value == "M" or precip_value == "T":
                        # "M" = Missing, "T" = Trace (keep as is)
                        precip_clean = precip_value
                    else:
                        try:
                            # Convert to float and back to string to clean format
                            precip_clean = str(float(precip_value))
                        except (ValueError, TypeError):
                            precip_clean = precip_value
                else:
                    precip_clean = ""
                    time_obs = ""
                    flags = ""
            else:
                precip_clean = ""
                time_obs = ""
                flags = ""

            csv_writer.writerow([
                station_name, acis_sid, date, precip_clean, time_obs, flags
            ])
        station_count += 1
        print(f"Downloaded: {station_name} (COOP: {coop_sid}, ACIS: {acis_sid})")
    else:
        print(f"No data for {stn.get('name', 'Unknown station')}")

    # Be respectful to the API
    time.sleep(0.1)

csv_file.close()

print(f"\nDone. Data saved for {station_count} stations.")
print(f"Skipped {skipped_no_coop} stations without COOP/ACIS IDs")
print(f"Skipped {skipped_out_of_bounds} stations outside bounding box")
print(f"Total stations processed: {len(stations)}")
print(f"Note: COOP stations without coordinates were included regardless of location")

Total stations found in NJ: 877

Sample station data:
Station 0: {'ll': [-75.0559, 40.5342], 'sids': ['US1NJHN0031 6', 'NJHN0031 10'], 'state': 'NJ', 'valid_daterange': [['2012-07-24', '2025-07-15']], 'name': 'FRENCHTOWN 0.5 N'}
Station 1: {'ll': [-74.83466, 40.37352], 'sids': ['US1NJMC0027 6', 'NJMC0027 10'], 'state': 'NJ', 'valid_daterange': [['2012-07-27', '2025-07-15']], 'name': 'HOPEWELL TWP 2.4 NW'}
Station 2: {'ll': [-74.79259, 40.27776], 'sids': ['US1NJMC0029 6', 'NJMC0029 10'], 'state': 'NJ', 'valid_daterange': [['2012-08-13', '2025-07-15']], 'name': 'EWING TWP 1.2 N'}
Downloaded: SOMERSET AIRPORT (COOP: SMQ, ACIS: 288174)
Downloaded: CHATHAM 2 W (COOP: CHTN4, ACIS: 281592)
Downloaded: SUSSEX AIRPORT (COOP: FWN, ACIS: 288645)
Downloaded: PHILADELPHIA/MT. HOLLY WFO (COOP: PHIN4, ACIS: 286964)
Downloaded: OCEAN CITY (COOP: OCAN4, ACIS: 286480)
Downloaded: SALEM (COOP: SALN4, ACIS: 287850)
Downloaded: CAPE MAY 2 NW (COOP: CPMN4, ACIS: 281351)
Downloaded: CAPE MAY CT HOUSE 4 N (CO